# Generator basics

CyriteHDL is internally driven by generator machinery, rather than  particular processing into a target representation by AST translation.

Take a simple conditional function in native Python:

In [1]:
import sys
sys.path.insert(0, "../../")

In [2]:
def native(a, b, z):
    if a == 1:
        b.next = 2
        z.next = True
    else:
        b.next = 0
        z.next = False

When this function is run, it determines at runtime when to branch, and not visit code inside condition statements that are not met. In order to translate this code to a target, a different internal representation is more practical:

In [3]:
def generator_func(context, a, b, z):
    yield (
        context.If(a == 1).Then(
            b.set(2), z.set(True)
        ).Else(
            b.set(0), z.set(False)
        )
    )

A specific caller is able to extract the entire branch tree from this construct, given a `context.If` class hierarchy that manages it. We create a simple context ad hoc:

In [4]:
from myirl.kernel.struct_cond import If

class AdHocContext:
    def __init__(self):
        self.If = If  

Then we iterate through the generation function and print out the statements in their debug-style representation:

In [5]:
from myirl import *

a, b = [ Signal(intbv()[6:], name = n) for n in "ab" ]
z = Signal(bool(), name = 'z')

context = AdHocContext()

def walk(context, func, emit, *args):
    for g in func(context, *args):
        emit(g)

def _emit(g):
    if isinstance(g, CondIterable):
        for s in reversed(list(g)):
            print(s)
        
walk(context, generator_func, _emit, a, b, z)

If (a == C:1)
Then {(b <= 2, z <= B:true)}
Then {(b <= 0, z <= B:false)}


A more complex target is a HDL language target instanced as follows:

In [6]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule(targets.VHDL)
walk(d, generator_func, lambda x: x.emit(d), a, b, z)

case a is
when "000001" =>
    b <= "000010";
    z <= '1';
when others =>
    b <= "000000";
    z <= '0';
end case;


## Conclusion

We note: There can be two representations of a Python routine. Since Python does neither allow us to use custom If..Else classes in its native dialect, nor can we override an assignment of a variable, we are a priori required to explicitely write down a dual representation for the same thing, if we want to execute and generate. The other, context sensitive option, is to use a decorator for a routine that does the following:
* AST translation within a generator context to a generator representation using `yield`
* Untouched execution of the function when in a native python execution context

## The myIRL internal representation

When decorating a top level function with a `@block` statement, its child funclets will be examined for specific decorator keywords and translation to the **IRL** (internal representation language) will occur.

This is a static construct and allows to emulate HDL python dialects such as MyHDL up to some extent.

Dynamic selection is provided by specific factory classes that are context sensitive. This means, depending on the target configuration, code is either emit to a HDL target, compiled directly into a hardware simulation or is run natively in conjunction with a simulation. This is also referred to as 'Co-Simulation'.

## When to use yield and when not

Because code is either translated or executed, it is not always transparent in which of the dual forms HDL code should be authored. In some cases it might be desirable to mix native code with procedural constructs in IRL notation.

The IRL provides generator constructs through decorators that work like statements inside a IRL sequence or process. In native form, an undecorated generator functions would be used using `yield from`. Inside a IRL statement list however, a decorated Generator object will be iterated through automatically. This can cause confusion, as shown with the `@hdlmacro` construct below.

First, we must import the augmented `Signal` class from the simulation module. This class is able to evaluate assignments behind the curtains and is thus usable for verification up to some extent.

In [128]:
from cyrite.simulation import Signal

@hdlmacro
def genfunc(a, b):
    print("RUN MACRO")
    yield [
        a.set(b),
        b.set(b + 1),
    ]
    print("FINISH MACRO")
    
def walk_sequence(a, b):
    
    a.next = 0
    b.next = 3
    # @hdlmacro is explicitely *called*
    genfunc(a, b)
    # This is not correct:
    # yield from genfunc(a, b)
    print(a, b)
   
    assert a == 3
    assert b == 4
    
    
    print("STOP")

In [129]:
a, b = [ Signal(intbv()[4:], name = n) for n in "ab" ]

walk_sequence(a, b)

RUN MACRO
FINISH MACRO
<a> : 0x3 <b> : 0x4
STOP


Because we're effectively executing `walk_sequence()` and no iteration is taking place on this level, genfunc() is called and **must** be decorated with `@hdlmacro`.  If we did not decorate, the macro in the `yield` form is actually not unrolling.

However, when we wish to generate code for external simulators, the game is different: In this case `walk_sequence()` would need to be a generator function and we might have to explicitely rewrite it - if we did not have an auxiliary: We can use said decorators to silently turn it into a generator under the hood.

## Code contexts

We can create decorators with dual use functionality, such that a function is *executed* in one context or *generates* in the other.

A hardware function, for example a IRL `@process`, always runs in parallel with other processes. A simulation specific sequence of stimuli again runs like a program with wait/delay statements.

A `@hdlmacro` is a priori a hardware-minded function which does not describe a timed simulation sequence, therefore it is a pure and explicit generator. When called within a sequential simulation context, it will work because the corresponding wrapper executes the generator function implicitely.

## Code verification - externally

The Cyrite emulation layer provides a `@sequence` decorator to automatically translate a sub-function into a generator.

However, there's a catch: When simply copying the `walk_sequence()` function, an external simulator will fail. Why? When assigning a `Signal` to a value, the value is not immediately valid, rather, a delta time step will have to be taken in order for the signal to update.

Therefore, we have to insert a `delay()` statement before each time we make use of a signals content. We construct a test bench `@block` with an adapted `@sequence`:

In [135]:
from cyhdl import *

@block
def testbench():
    a, b = [ Signal(intbv()[4:], name = n) for n in "ab" ]

    @sequence
    def main():
        a.next = 0
        b.next = 3
        
        yield delay(1) # Wait to settle
        
        # @hdlmacro is explicitely *called*
        # genfunc(a, b)
        # This is allowed in this context:
        yield from genfunc(a, b)

        yield delay(1) # Again, wait to settle signal values
        print(a, b)
        
        assert a == 3
        assert b == 4

        print("STOP")

    return instances()

Note that we can use the `yield from` notation here. It is silently translated into a call inside the resulting yield sequence. We can display this by calling `.unparse()`:

In [138]:
print(testbench.unparse())

Unparsing unit testbench


@block
def testbench():
    (a, b) = [Signal(intbv()[4:], name=n) for n in 'ab']

    @sequential
    def main(_sequence):
        _sequence += [a.set(0), b.set(3), wait(delay(1)), genfunc(a, b), wait(delay(1)), print_(a, b), assert_((a == 3), 'Failed in /tmp/ipykernel_18868/1627005758.py:testbench():22'), assert_((b == 4), 'Failed in /tmp/ipykernel_18868/1627005758.py:testbench():23'), print_('STOP')]
    return instances()



In [136]:
from myirl.test.common_test import Simulator
tb = testbench()

s = Simulator(targets.VHDL)
s.run(tb, 200, debug = True)

RUN MACRO
FINISH MACRO
DEBUG: CALL MACRO (VOID) [Macro 'genfunc']
 Writing 'testbench' to file /tmp/testbench.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance testbench I/F: [// ID: testbench_0 to testbench]] /tmp/myirl_testbench_72b2grat/
==== COSIM stdout ====
0x3 0x4
STOP



0

The simulation hence passes, however, try removing a `delay()` statement and it will fail. This behaviour is always wanted in the simulation world. Because our simulation signals are initialized by a dummy simulation signal internally and receive their assignments immediately, the behaviour is *not* the same.

Finally, we may want to have a look at the transpiled resulting VHDL file:

In [113]:
!cat {s.used_files[0]}

-- File generated from source:
--     ../../myirl/emulation/myhdl.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity testbench is
end entity testbench;

architecture myhdl_emulation of testbench is
    -- Local type declarations
    -- Signal declarations
    signal a : unsigned(3 downto 0);
    signal b : unsigned(3 downto 0);
begin
    
main:
    process
    begin
        a <= x"0";
        b <= x"3";
        wait for 1 ns;
        a <= b;
        b <= resize((resize(b, 5) + 1), 4);
        wait for 1 ns;
        print("0x"& hstr(a) & " " & "0x"& hstr(b));
        assert (a = x"3")
            report "Failed in /tmp/ipykernel_18868/1223585289.py:testbench():22" severity failure;
        assert (b = x"4")
            report "Failed in /tmp/ipykernel_18868/1223585289.py:testbench():23" severity failu

### Hardware generation versus simulation

The classical HDL languages don't explicitely differentiate between constructs that translate to real hardware elements or are mainly meant for simulation.

One could thus be tempted to insert `delay()` specifications into a `@hdlmacro` generator in order to simplify the main() sequence.
However this is a deprecated method, as `@hdlmacro` constructs are meant to create hardware in particular.

A Python simulation routine however makes the distinction between time steps and events versus hardware elements by the `yield` statement. To keep hardware only and simulation constructs separate, specific decorators are used for sequential statements, such as `@cyrite_method.sequence`.

See also [Factory class: HDL macros](../notebooks/factory_class_arch.ipynb#HDL-macros) for some details on various context sensitive macro approaches.